In [ ]:
from model import run_model
from sklearn.model_selection import train_test_split
from pathlib import Path

In [ ]:
TARGET = 'is_fit'

# Find all CSV files in subfolders of data/
data_dir = Path('data')
csv_files = [f for f in data_dir.rglob('*.csv') if f.parent != data_dir]

# Dictionary to store results
results = {}

# Iterate over each CSV file
for csv_file in csv_files:
    filename = str(csv_file)
    
    try:
        ROC_AUCs = run_model(filename, TARGET)
        results[filename] = ROC_AUCs
    except Exception as e:
        print(f"Error processing {filename}: {e}")
        results[filename] = None

for filename, roc_aucs in results.items():
    print(f"{filename}: {roc_aucs}")

In [ ]:
import re
import matplotlib.pyplot as plt

# Parse results and organize by method
data_by_method = {}

for filename, roc_aucs in results.items():
    # Extract method and percentage using regex
    # Pattern: data/fitness/{method}/minority_{percentage}pct...
    match = re.search(r'data/fitness/([^/]+)/minority_(\d+)pct', filename)
    
    if match and roc_aucs:
        method = match.group(1)
        percentage = int(match.group(2))
        roc_auc = roc_aucs[0]  # Take first index as requested
        
        if method not in data_by_method:
            data_by_method[method] = {'percentages': [], 'roc_aucs': []}
        
        data_by_method[method]['percentages'].append(percentage)
        data_by_method[method]['roc_aucs'].append(roc_auc)

# Sort each method's data by percentage
for method in data_by_method:
    # Zip, sort, and unzip to keep percentages and roc_aucs aligned
    sorted_pairs = sorted(zip(data_by_method[method]['percentages'], 
                             data_by_method[method]['roc_aucs']))
    data_by_method[method]['percentages'], data_by_method[method]['roc_aucs'] = zip(*sorted_pairs)

# Plot each method with different color
plt.figure(figsize=(10, 6))

for method, data in data_by_method.items():
    plt.plot(data['percentages'], data['roc_aucs'], marker='o', label=method)

plt.xlabel('Minority Percentage (%)')
plt.ylabel('ROC AUC')
plt.title('ROC AUC vs Minority Percentage by Method')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()